In [1]:
from loglead.loaders.bgl import BGLLoader
from loglead.enhancers import EventLogEnhancer, SequenceEnhancer
import polars as pl


In [ ]:
bgl_log_path = "./Datasets/bgl/BGL.log"
bgl_loader = BGLLoader(filename=bgl_log_path)
df = bgl_loader.execute()
df = df.fill_null("")

In [ ]:
enhancer = EventLogEnhancer(df)
df = enhancer.normalize()
df = enhancer.parse_spell()
df = enhancer.words()


In [ ]:
df = enhancer.trigrams()
df = enhancer.parse_drain()
df = enhancer.parse_tip()


In [ ]:
df = enhancer.parse_pliplom()

In [ ]:
df = enhancer.parse_iplom()

In [ ]:
df = enhancer.parse_brain()

In [ ]:
df = enhancer.length()

In [2]:
df = pl.read_parquet("output.parquet")

In [3]:
from loglead.anomaly_detection import AnomalyDetector
anomaly_detector = AnomalyDetector(
    item_list_col="e_words",
    numeric_cols=["e_words_len", "e_trigrams_len", "e_chars_len", "e_lines_len", "e_event_id_len"],
    label_col="anomaly",
    store_scores=True,
    print_scores=True,
    auc_roc=True
)

In [4]:
anomaly_detector.test_train_split(df, test_frac=0.9, shuffle=True)


In [5]:
anomaly_detector.train_IsolationForest(n_estimators=100, contamination="auto")



In [6]:
df = anomaly_detector.predict()


Results from model: IsolationForest
Accuracy: 0.9265
F1 Score: 0.0000
AUC-ROC Score: 0.6949
Confusion Matrix:
[[3959272       0]
 [ 313894       0]]
AUCROC: 0.6949


In [ ]:
def split_parquet_polars(df, output_prefix, chunk_size):
    for i, chunk in enumerate(range(0, len(df), chunk_size)):
        df.slice(chunk, chunk_size).write_parquet(f"{output_prefix}_{i}.parquet")

split_parquet_polars(df, "anomaly_results", 1000000)



In [9]:
import pandas as pd
df.sample(n=1000, shuffle=True).to_pandas().to_html("anomaly_results_sample.html")